# 1 - Import Dependencies

In [1]:
!pip install stable-baselines3

In [2]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2 - Load Environment

In [3]:
environment_name = 'CartPole-v0'
env = gym.make(environment_name)

In [4]:
environment_name

'CartPole-v0'

In [5]:
episodes = 5
for episode in range(1, episodes  + 1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episodes:{} Score:{}'.format(episode, score))
env.close()

Episodes:1 Score:18.0
Episodes:2 Score:16.0
Episodes:3 Score:21.0
Episodes:4 Score:17.0
Episodes:5 Score:13.0


# 3 - Understanding The Environment

In [6]:
env.action_space

Discrete(2)

In [7]:
env.action_space.sample()

0

In [8]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [9]:
env.observation_space.sample()

array([ 1.6415328e+00,  3.2384878e+37,  2.7003181e-01, -1.3490027e+38],
      dtype=float32)

# 4 - Train an RL Model

In [10]:
log_path = os.path.join('Training', 'Logs')

In [11]:
log_path

'Training\\Logs'

In [12]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device


In [13]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_3
-----------------------------
| time/              |      |
|    fps             | 321  |
|    iterations      | 1    |
|    time_elapsed    | 6    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 319         |
|    iterations           | 2           |
|    time_elapsed         | 12          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008946226 |
|    clip_fraction        | 0.114       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.000727   |
|    learning_rate        | 0.0003      |
|    loss                 | 7.1         |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0189     |
|    value_loss           | 52          |
-----------------------------------------
---

# 5 - Save and Reload Model 

In [14]:
PPO_Path = os.path.join('Training', 'Saved Models', 'PPO_Model_Cartpole')

In [15]:
model.save(PPO_Path)

In [16]:
del model

In [17]:
model = PPO.load(PPO_Path, env=env)

# 6 - Evaluation

In [18]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

C:\Users\dilan\anaconda3\envs\deep-learning\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

In [19]:
env.close()

# 7 - Test Model

In [20]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    print('Episodes:{} Score:{}'.format(episode, score))
env.close()

Episodes:1 Score:[200.]
Episodes:2 Score:[200.]
Episodes:3 Score:[200.]
Episodes:4 Score:[200.]
Episodes:5 Score:[148.]


# 8 - Viewing Logs in Tensorboard

In [21]:
training_log_path = os.path.join(log_path, 'PPO_2')

In [22]:
!tensorboard --logdir={training_log_path}

^C


# 9 - Adding a callback to the Training Stage

In [23]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [24]:
save_path = os.path.join('Training', 'Saved Models')

In [27]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=True)
eval_callback = EvalCallback(env,
                            callback_on_new_best=stop_callback,
                            eval_freq=10000,
                            best_model_save_path=save_path,
                            verbose=True)

In [29]:
model = PPO('MlpPolicy', env, verbose=True, tensorboard_log=log_path)

Using cuda device


In [31]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_4
-----------------------------
| time/              |      |
|    fps             | 605  |
|    iterations      | 1    |
|    time_elapsed    | 3    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 476         |
|    iterations           | 2           |
|    time_elapsed         | 8           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008804232 |
|    clip_fraction        | 0.103       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00106    |
|    learning_rate        | 0.0003      |
|    loss                 | 7.47        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0147     |
|    value_loss           | 51.4        |
-----------------------------------------
---

# 10 - Changing Policies

In [32]:
net_arch = [dict(pi=[128,128,128,128], vf=[128,128,128,128])]

In [33]:
model = PPO('MlpPolicy', env, verbose=True, tensorboard_log=log_path, policy_kwargs={'net_arch':net_arch})

Using cuda device


In [34]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_5
-----------------------------
| time/              |      |
|    fps             | 455  |
|    iterations      | 1    |
|    time_elapsed    | 4    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 331         |
|    iterations           | 2           |
|    time_elapsed         | 12          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.013980968 |
|    clip_fraction        | 0.201       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.681      |
|    explained_variance   | -0.0183     |
|    learning_rate        | 0.0003      |
|    loss                 | 3.47        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0228     |
|    value_loss           | 19          |
-----------------------------------------
---

# 10 - Using an Alternate Algorithm

In [35]:
from stable_baselines3 import DQN

In [36]:
model = DQN('MlpPolicy', env, verbose=True, tensorboard_log=log_path)

Using cuda device


In [37]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.963    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 5500     |
|    time_elapsed     | 0        |
|    total_timesteps  | 77       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.932    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 6505     |
|    time_elapsed     | 0        |
|    total_timesteps  | 143      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.903    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 6630     |
|    time_elapsed     | 0        |
|    total_timesteps  | 205      |
----------------------------------
------------------------